<a href="https://colab.research.google.com/github/shubhammanuwas/shubham-ai/blob/main/tree_width.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch torchvision opencv-python pillow segment-anything


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


In [ ]:
!pip install -q 'git+https://github.com/facebookresearch/segment-anything.git'

  Preparing metadata (setup.py) ... done


In [ ]:
!mkdir -p {HOME}/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth -P {HOME}/weights

In [ ]:
import os

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_b_01ec64.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

/content/weights/sam_vit_b_01ec64.pth ; exist: True


In [ ]:
import os
from PIL import Image

# Define the path to the images folder
images_folder = "/content/images"

# Get all image paths from the folder
image_paths = [os.path.join(images_folder, file) for file in os.listdir(images_folder) if file.endswith(('.jpeg', '.jpg', '.png'))]

# Load all images into a list
images = [Image.open(image_path) for image_path in image_paths]

print(f"Loaded {len(images)} images.")

Loaded 10 images.


In [ ]:
from PIL import Image, ImageDraw
import numpy as np
import torch
import cv2
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

# Load the image using a relative path
image_path = "/content/images/tree3.jpeg"  # Assuming the image is in the "images" folder in your working directory
image = Image.open(image_path)
image_np = np.array(image)

# The rest of your code remains the same
print(f"Image path:{image_path}")
print(f"Image size: {image.size}")
print(f"Image Width: {image.width}")

# Step 2: Load the SAM model


DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

MODEL_TYPE = "vit_b"
sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)


# Step 3: Generate masks using SAM
mask_generator = SamAutomaticMaskGenerator(sam)
masks = mask_generator.generate(image_np)

# Step 4: Identify the tree trunk (largest segmented object)
largest_mask = max(masks, key=lambda x: x['area'])['segmentation']

# Step 5: Calculate the bounding box of the tree trunk
x, y, w, h = cv2.boundingRect(largest_mask.astype(np.uint8))
tree_trunk_pixel_width = w  # Width of the tree trunk in pixels

# Step 6: Convert pixel width to real-world measurement (in cm)
focal_length_cm = 2.4  # Focal length in cm
object_distance_cm = 50  # Distance to the object in cm
sensor_width_cm = 0.756  # Sensor width in cm

# Calculate the focal length in pixels
focal_length_pixels = (focal_length_cm * image.width) / sensor_width_cm

# Calculate the real-world width in cm using the corrected formula
real_world_width_cm = (tree_trunk_pixel_width * object_distance_cm) / focal_length_pixels

# Step 7: Estimate the diameter (assuming the tree trunk is roughly circular)
tree_trunk_diameter_cm = real_world_width_cm

# Print the results
print(f"Image Dimensions: {image.width} x {image.height}")
print(f"Estimated Pixel Width of Tree Trunk: {tree_trunk_pixel_width} pixels")
print(f"Estimated Real-World Diameter of Tree Trunk: {tree_trunk_diameter_cm:.2f} cm")

# Step 8: Visualize the segmentation and the diameter line
# Convert the mask to an image
mask_image = Image.fromarray((largest_mask * 255).astype(np.uint8))

# Create an overlay of the mask on the original image
overlay = Image.fromarray(cv2.addWeighted(image_np, 0.5, np.array(mask_image.convert('RGB')), 0.5, 0))

# Draw the bounding box and the diameter line on the overlay
draw = ImageDraw.Draw(overlay)
draw.rectangle([x, y, x + w, y + h], outline="red", width=3)  # Bounding box in red
draw.line([x, y + h//2, x + w, y + h//2], fill="blue", width=3)  # Diameter line in blue

# Show the result
overlay.show()

# Optionally, save the image with markings
overlay.save("/content/images/marked_tree3.jpeg")


Image path:/content/images/tree3.jpeg
Image size: (960, 1280)
Image Width: 960
Image Dimensions: 960 x 1280
Estimated Pixel Width of Tree Trunk: 687 pixels
Estimated Real-World Diameter of Tree Trunk: 11.27 cm


In [13]:
from sklearn.model_selection import train_test_split

# Split the images and paths into training and testing sets
train_images, test_images, train_paths, test_paths = train_test_split(images, image_paths, test_size=0.2, random_state=42)

print(f"Training on {len(train_images)} images, testing on {len(test_images)} images.")

Training on 8 images, testing on 2 images.


In [ ]:
# Assuming the SAM model is already loaded and configured as per your code above
diameters = []

for image in train_images:
    image_np = np.array(image)

    # Generate masks using SAM
    masks = mask_generator.generate(image_np)

    # Identify the tree trunk (largest segmented object)
    largest_mask = max(masks, key=lambda x: x['area'])['segmentation']

    # Calculate the bounding box of the tree trunk
    x, y, w, h = cv2.boundingRect(largest_mask.astype(np.uint8))
    tree_trunk_pixel_width = w  # Width of the tree trunk in pixels

    # Calculate the real-world width in cm using the corrected formula
    real_world_width_cm = (tree_trunk_pixel_width * object_distance_cm) / focal_length_pixels

    # Estimate the diameter (assuming the tree trunk is roughly circular)
    tree_trunk_diameter_cm = real_world_width_cm

    diameters.append(tree_trunk_diameter_cm)

print("Estimated diameters:", diameters)


In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Create the input features (pixel widths) and target values (diameters)
X_train = np.array([cv2.boundingRect(max(mask_generator.generate(np.array(image)), key=lambda x: x['area'])['segmentation'].astype(np.uint8))[2] for image in train_images]).reshape(-1, 1)
y_train = np.array(diameters)

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict the diameters for the test images
X_test = np.array([cv2.boundingRect(max(mask_generator.generate(np.array(image)), key=lambda x: x['area'])['segmentation'].astype(np.uint8))[2] for image in test_images]).reshape(-1, 1)
y_pred = model.predict(X_test)

print("Predicted diameters:", y_pred)


In [ ]:
import matplotlib.pyplot as plt

# Plot the true vs predicted diameters
plt.figure(figsize=(10, 5))
plt.scatter(X_test, y_pred, color='blue', label='Predicted Diameter')
plt.xlabel('Pixel Width of Tree Trunk')
plt.ylabel('Diameter (cm)')
plt.title('Linear Regression: Predicted Diameter vs Pixel Width')
plt.legend()
plt.show()
